# Get started with Snowflake and the Hopsworks Feature Store
This tutorial notebook will help you to get started working with the Hopsworks feature store and Snowflake.

- We assume that you already have snowflake account. If not please follow the [tutorial](https://www.snowflake.com/webinar/virtual-hands-on-labs/virtual-hands-on-lab-2020-03-18/) to setup your Snowflake account and load telcom churn dataset   

## Create Snowflake connector in Hopsworks <a name="access_snowflake"></a>

#### Step1 ) From Hopsworks go to Feature Store select Storage Connectors and click Create New
![1.jpg](images/hopsworks/connector/1.png)

    
#### Step2 ) Fill in name and description of the connector. <br>  

 &nbsp; In JDBC connection string provide the connection and credentials required to connect to your Snowflake account. 

**NOTE:** For Snowflake accounts in regions other than US WEST add the Region ID after a period <ACCOUNT>.<REGION ID> i.e. ra96958.eu-central-1. 
 
                                                                                     
 &nbsp; then click Create
![2.jpg](images/hopsworks/connector/2.png)


## Python

In [ ]:
import hsfs

connection = hsfs.connection()

# get a reference to the feature store, you can access also shared feature stores by providing the feature store name
fs = connection.get_feature_store()
connector = fs.get_storage_connector("snowflake_spark_connector", "JDBC")

In [ ]:
import snowflake.connector # tested on snowflake-connector-python==2.3.1 
# Connecting to Snowflake using the default authenticator
ctx = snowflake.connector.connect(
  user='HOPSWORKS',
  password=connector.arguments,
  account='ra96958.eu-central-1',
  warehouse='HOPSWORKS_WH',
  database='ML_WORKSHOP',
  schema='PUBLIC'
)

In [ ]:
import pandas as pd
# Query Snowflake Data
cs=ctx.cursor()
allrows=cs.execute("""select Cust_ID,STATE,ACCOUNT_LENGTH,AREA_CODE,PHONE,INTL_PLAN,VMAIL_PLAN,VMAIL_MESSAGE,
                   DAY_MINS,DAY_CALLS,DAY_CHARGE,EVE_MINS,EVE_CALLS,EVE_CHARGE,NIGHT_MINS,NIGHT_CALLS,
                   NIGHT_CHARGE,INTL_MINS,INTL_CALLS,INTL_CHARGE,CUSTSERV_CALLS,
                   CHURN from CUSTOMER_CHURN """).fetchall()

churn = pd.DataFrame(allrows)
churn.columns=['Cust_id','State','Account Length','Area Code','Phone','Intl Plan', 'VMail Plan', 'VMail Message','Day Mins',
            'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls','Night Charge',
            'Intl Mins','Intl Calls','Intl Charge','CustServ Calls', 'Churn?']

pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
churn

## Spark
#### Step 1: Attache snowflake-jdbc-3.12.8.jar to the cluster.
#### https://mvnrepository.com/artifact/net.snowflake/snowflake-jdbc/3.12.8
#### Step 2: Attache spark-snowflake_2.11-2.8.1 driver to the cluster.
#### https://mvnrepository.com/artifact/net.snowflake/spark-snowflake_2.11/2.8.1-spark_2.4

In [ ]:
import com.logicalclocks.hsfs._
import scala.collection.JavaConversions._
import collection.JavaConverters._

val connection = HopsworksConnection.builder().build();
val fs = connection.getFeatureStore();
val connector = fs.getStorageConnector("snowflake_spark_connector",StorageConnectorType.JDBC)

In [ ]:
val SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"
val sfOptions = Map(
"sfURL" -> connector.getConnectionString(),
"sfUser" -> "HOPSWORKS",
"sfPassword" -> connector.getArguments(),
"sfDatabase" -> "ML_WORKSHOP",
"sfSchema" -> "PUBLIC",
"sfWarehouse" -> "HOPSWORKS_WH",
"sfRole" -> "HOPSWORKS_ROLE"
)

val df = spark.read.
format(SNOWFLAKE_SOURCE_NAME).
options(sfOptions).
option("dbtable", "CUSTOMER_CHURN").
load()